In [1]:
from matplotlib import pyplot as plt
import numpy as np
import pymc3 as pm
import scipy as sp
import seaborn as sns
from theano import tensor as tt
import pandas as pd

In [3]:
class Error(pm.Discrete):
    def __init__(theta, alpha, beta, *args, **kwargs):
        super(Error, self).__init(*args, **kwargs)
        self.theta = theta
        self.alpha = alpha
        self.beta = beta

    def random(self, point=None, size=None):        
        return 1

    def logp(self, value):        
        return 0

In [4]:
def stick_breaking(beta):
    p = tt.concatenate([[1], tt.extra_ops.cumprod(1 - beta)[:-1]])
    return beta * p

In [ ]:
with pm.Model() as model:
    alpha = pm.Gamma('alpha', 1., 1.)
    beta = pm.Beta('beta', 1., alpha, shape=K)
    w = pm.Deterministic('w', stick_breaking(beta))

    tau = pm.Gamma('tau', 1., 1., shape=K)
    lambda_ = pm.Uniform('lambda', 0, 5, shape=K)
    mu = pm.Normal('mu', 0, tau=lambda_ * tau, shape=K)
    obs = pm.Mixture('obs', w, mu, tau=lambda_ * tau,
                           observed=old_faithful_df.std_waiting.values)
